# Part I. One-sided finite differences

Write a function, `deriv`, which computes a derivative of its argument at a given point, $x$, using a one-sided finite difference rule with a given step side $h$, with the approximation order of $O(h^2)$. 

In [1]:
def deriv(f, x, h):
    """ Compute a derivative of `f` at point `x` with step size `h`.
    
    Compute the derivative using the one-sided rule of the approximation order of $O(h^2)$.
    
    Parameters
    ----------
    f : callable
        The function to differentiate
    x : float
        The point to compute the derivative at.
    h : float
        The step size for the finite different rule.
        
    Returns
    -------
    fder : derivative of f(x) at point x using the step size h.
    """
    x1 = x + h
    dx = x1 - x
    df = f(x1) - f(x)
    fder = df / dx
    return fder

#### Test I.1

Test your function on a simple test case: differentiate $f(x) = x^3$ at $x=0$. Comment on whether your results are consistent with the expected value of $f'(x) = 0$ and on an expected scaling with $h\to 0$.

 (10% of the total grade)

In [2]:
x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(lambda x: x**3, x, h)
    print("%5f -- %7.4g" % (h, err))

0.010000 --  0.0001
0.001000 --   1e-06
0.000100 --   1e-08
0.000010 --   1e-10


The results are consistent with what we'd expect for h approaching 0

### Test I.2

Now use a slightly more complicated function, $f(x) = x^2 \log{x}$, evaluate the derivative at $x=1$ using your one-sided rule and a two-point one-sided rule. Roughly estimate the value of $h$ where the error stops decreasing, for these two schemes. 
(15% of the total grade)

In [5]:
from math import log

def f(x):
    return x**2 * log(x)
    
def fder(x):
    return x * (2.*log(x) + 1)

In [12]:
def deriv2pt(f, x, h):
    x1 = x + h
    dx = x1 - x
    df = f(x1) - f(x)
    fder = df / dx
    return fder

print("Actual ", actual)
for h in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]:
    err = deriv(f, 1, h)
    actual = fder(1)
    print("h value: ", h)
    print("Computed: ", err)
    print("Error: ", actual - err)
   

Actual  1.0
h value:  0.01
Computed:  1.0150332503316761
Error:  -0.015033250331676129
h value:  0.001
Computed:  1.0015003332500332
Error:  -0.0015003332500331812
h value:  0.0001
Computed:  1.00015000333325
Error:  -0.00015000333324999282
h value:  1e-05
Computed:  1.0000150000333332
Error:  -1.5000033333212315e-05
h value:  1e-06
Computed:  1.0000015000003333
Error:  -1.5000003332765743e-06
h value:  1e-07
Computed:  1.0000001500000033
Error:  -1.5000000330722685e-07
h value:  1e-08
Computed:  1.000000015
Error:  -1.4999999908837935e-08
h value:  1e-09
Computed:  1.0000000015000001
Error:  -1.5000001241105565e-09
h value:  1e-10
Computed:  1.00000000015
Error:  -1.5000001241105565e-10
h value:  1e-11
Computed:  1.000000000015
Error:  -1.5000001241105565e-11
h value:  1e-12
Computed:  1.0000000000015001
Error:  -1.5001333508735115e-12
h value:  1e-13
Computed:  1.0000000000001499
Error:  -1.4988010832439613e-13
h value:  1e-14
Computed:  1.000000000000015
Error:  -1.509903313490213e-

### Test I.3 

Now try differentiating $x^2 \log(x)$ at $x=0$. Use the three-point one-sided rule. Note that to evaluate the function at zero, you need to special-case this value. Check the scaling of the error with $h$, explain your results. 
(25% of the total grade)

In [13]:
def f(x):
    if x == 0:
        # the limit of $x^2 log(x)$ at $x-> 0$ is zero, even though log(x) is undefined at x=0
        return 0.0
    else:
        return x**2 * log(x)
    
def fder(x):
    if x == 0:
        return 0.0
    else:
        return x*(2*log(x) + 1)

x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(f, x, h) - fder(x)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.04605
0.001000 -- -0.006908
0.000100 -- -0.000921
0.000010 -- -0.0001151


The error decreases as expected as the value of h approaches zero.

# Part II. Midpoint rule 

Write a function which computes a definite integral using the midpoint rule up to a given error, $\epsilon$. Estimate the error by comparing the estimates of the integral at $N$ and $2N$ elementary intervals. 

In [17]:
import numpy as np

b = 5
a = 0
N = 5
h = (b - a) / N
x_list = [a + 0.5 * h + n * h for n in range(N)]

def midpoint_rule(func, a, b, eps):
    """ Calculate the integral of f from a to b using the midpoint rule.
    
    Parameters
    ----------
    func : callable
        The function to integrate.
    a : float
        The lower limit of integration.
    b : float
        The upper limit of integration.
    eps : float
        The target accuracy of the estimate.
        
    Returns
    -------
    integral : float
        The estimate of $\int_a^b f(x) dx$.
    """
    N = 1
    h = (b-a) / N
    x_list = [a + 0.5 * h + n * h for n in range(N)]
    y_list = np.array([func(x) for x in x_list])
    new_est = h * np.sum(y_list)
    old_est = new_est + 1e5
    while(np.abs(new_est-old_est)>eps):
        N = N * 2
        h = (b - a) / N
        x_list = [a + 0.5 * h + n * h for n in range(N)]
        y_list = np.array([func(x) for x in x_list])
        old_est = new_est
        new_est = h * np.sum(y_list)
    return new_est

### Test II.1

Test your midpoint rule on a simple integral, which you can calculate by paper and pencil.

Compare the rate of convergence to the expected $O(N^{-2})$ scaling by studying the number of intervals required for a given accuracy $\epsilon$.

Compare the numerical results to the value you calculated by hand. Does the deviation agree with your estimate of the numerical error?
(20% of the total grade)


Functions as intended. 

In [18]:
def f2(x):
    return x**2

midpoint_rule(f2, 1, 5, eps=1e-5)

41.33333206176758

### Test II.2

Now use your midpoint rule to compute the value of

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx
$$

up to a predefined accuracy of $\epsilon=10^{-4}$.

Note that the integral contains an integrable singularity at the lower limit. Do calculations two ways: first, do a straightforward computation; next, subtract the singularity. Compare the number of iterations required to achieve the accuracy of $\epsilon$.

(30% of the total grade)

In [20]:
def f2(x):
    return np.sin(np.sqrt(x)) / x

def f3(x):
    return f2(x) - 1 / np.sqrt(x)

In [21]:
midpoint_rule(f3, 0, 1, eps=1e-4)

-0.10788699436045195